<a href="https://colab.research.google.com/github/christiane-villaroel/credit-card-fraud-detection/blob/main/real_connection_part%202%20into%20mySQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mysql-connector-python


In [2]:
import mysql.connector
from mysql.connector import Error
import pandas as pd



In [3]:
config = {
    'host': 'localhost',
    'user': 'root',
    'password': 'Andrewcool318',
    'database': 'dailytest'
}

In [4]:
csv_file_path = '/content/cleaned_data2.0.csv'


In [5]:
# Sample column definitions based on the CSV file
column_definitions = {
    'step': 'INT',
    'type': 'VARCHAR(255)',
    'amount': 'FLOAT',
    'nameOrig': 'VARCHAR(255)',
    'oldbalanceOrg': 'FLOAT',
    'newbalanceOrig': 'FLOAT',
    'nameDest': 'VARCHAR(255)',
    'oldbalanceDest': 'FLOAT',
    'newbalanceDest': 'FLOAT',
    'isFraud': 'INT',
    'isFlaggedFraud': 'INT'
}


In [6]:
# Create the table creation SQL query
def create_table_query(table_name, columns):
    cols = ",\n".join([f"{col_name} {data_type}" for col_name, data_type in columns.items()])
    query = f"CREATE TABLE IF NOT EXISTS {table_name} (\n{cols}\n);"
    return query


In [7]:
df = pd.read_csv(csv_file_path)

In [8]:
# Connect to MySQL database
try:
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

    # Create the table
    table_name = 'log_data'
    query = create_table_query(table_name, column_definitions)
    cursor.execute(query)

    # Insert DataFrame into MySQL table using batch inserts
    batch_size = 1000  # You can adjust this batch size based on your memory capacity and performance
    for start in range(0, len(df), batch_size):
        end = start + batch_size
        batch = df.iloc[start:end]

        insert_query = f"""
        INSERT INTO {table_name} ({', '.join(column_definitions.keys())})
        VALUES ({', '.join(['%s'] * len(column_definitions))});
        """

        cursor.executemany(insert_query, [tuple(row) for row in batch.itertuples(index=False, name=None)])

    connection.commit()

except Error as e:
    print(f"Error: {e}")
finally:
    if 'connection' in locals() and connection.is_connected():
        cursor.close()
        connection.close()
print("CSV data has been imported into MySQL database successfully.")


Error: 2003 (HY000): Can't connect to MySQL server on 'localhost:3306' (99)
CSV data has been imported into MySQL database successfully.
